In [ ]:
# Advanced Statistical Machine Learning & Pattern Recognition - CO495
# skeleton for CW1
# Written for Python 3.

In [ ]:
import h5py
from scipy.io import savemat
import numpy as np
import csv
import matplotlib.pyplot as plt

try:
    %matplotlib inline
except NameError:
    # this is a terminal, that's fine.
    pass

In [ ]:
# load the file.
f = h5py.File('X.mat')
f.keys()

X = f.get('X').value

# optionally visualise the X matrix.
try:
    plt.plot(data[0, :], data[1, :], '*')
except:
    pass

# EM Init

In [ ]:
def EMInit(B, C):
    A = []
    print('Please fill in the function and delete the comment!')
    return A

# EM E step

In [ ]:
def EMEStep(B, C, D):
    E = []
    print('Please fill in the function and delete the comment!')
    return E

# EM M step

In [ ]:
def EMMStep(B, C, E):
    A = []
    print('Please fill in the function and delete the comment!')
    return A

# GMM Demo

In [ ]:
# aux functions
def random_params_generator(K):
    params = {}
    for i in range(1, K + 1):
        nn = 'means_{}'.format(i)
        params[nn] = np.random.rand(2)
        nn = 'covars_{}'.format(i)
        params[nn] = np.random.rand(2, 2)
        nn = 'mixCoeff_{}'.format(i)
        params[nn] = np.random.rand(1)
    return params


def convert_params_to_row(params, K):
    # accepts the params dict and converts it to a row to be exported.
    # The covars are flattened per row.
    row = []
    for var in ['means', 'covars', 'mixCoeff']:
        for i in range(1, K + 1):
            elems = params['{}_{}'.format(var, i)].flatten()
            row += [str(el) for el in elems]
    return row

In [ ]:
def GMMDemo(X, K=4):
    # accepts a 2 dimensional matrix X with the observed 
    # data and the K, which represents the number of clusters.
    # Please do not modify it!
    #
    # initialise the parameters
    params = EMInit(X, K) 
    # params should be a dict() with the following format:
    # params['means_[i]'] -> [2x1 float] (i \in [1, K])
    # params['covars_[i]'] -> [2x2 float] (i \in [1, K])
    # params['mixCoeff_[i]'] -> [1x1 float] (i \in [1, K])
    
    # EM algorithm
    # do not change the iterations --- keep them fixed.
    for i  in range(10):
        # E step
        resp = EMEStep(X, K, params); # compute responsibilities, i.e., every \gamma(z_{n,k})

        # M step
        params = EMMStep(X, K, resp) # update the values for the parameters

    # comment out the line below once you have implemented 
    # the init code!
    params = random_params_generator(K)
    # save the final values for the parameters
    # savemat('params.mat', {'params' : params})
    # export in csv.
    row = convert_params_to_row(params, K)
    with open('params.csv', 'w', encoding='utf-8-sig') as fn:
        writer = csv.writer(fn)
        writer.writerow(row)

In [ ]:
GMMDemo(X)